In [1]:
# #pd.concat([df1[~df1.index.isin(df2.index)], df2])
# import pandas as pd
# import numpy as np

# df1 = pd.read_excel('teil1.xlsx')
# df2 = pd.read_excel('teil2.xlsx')
# df1.set_index('Transfer Order Number', inplace=True)
# df2.set_index('Transfer Order Number', inplace=True)


In [2]:
# df1 = pd.concat([df1[~df1.index.isin(df2.index)], df2])


In [3]:

import pandas as pd
import numpy as np
from SQL import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder
#from data_Class.SQL import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder
    
dfOrder = sql_datenLadenOder()
dfOrderItems = sql_datenLadenOderItems()
dfLabel = sql_datenLadenLabel()
dfStammdaten = sql_datenLadenStammdaten()

In [4]:

        
dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
for each in dfStammdaten.index:
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'CS':
        dfStammdaten.loc[each, 'CS'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'CS'] = 0        
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'OUT':
        dfStammdaten.loc[each, 'OUT'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'OUT'] = 0
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'D97':
        dfStammdaten.loc[each, 'PAL'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'PAL'] = 0       
#dfStammdaten.head(10)

In [5]:

dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].astype(str)
dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].str.replace('0000000000', '')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

dfOrderItems['O'] = dfOrderItems['Outers'] * dfOrderItems['OUT']
dfOrderItems['Picks PAL'] = dfOrderItems.O / dfOrderItems.PAL
dfOrderItems['Picks CS'] = dfOrderItems.O / dfOrderItems.CS
dfOrderItems['Picks OUT'] = dfOrderItems.O/ dfOrderItems.OUT
#Bereinige Berechnungen der Picks 
for i in range(0,len(dfOrderItems.index)):
    #----PAL bereinigen
        if (dfOrderItems.loc[i,'Picks PAL'] <1):
            dfOrderItems.loc[i,'Picks PAL'] = 0
    #----cs bereinigen
        if (dfOrderItems.loc[i,'Picks CS'] <1):
            dfOrderItems.loc[i,'Picks CS'] = 0 
    #mögliche PAL picks abziehen
        if (dfOrderItems.loc[i,'Picks PAL'] >=1):
            dfOrderItems.loc[i,'Picks CS'] = (dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks PAL'] * dfOrderItems.loc[i,'PAL'])) * dfOrderItems.loc[i,'CS']
    #---OUT bereinigen
        if (dfOrderItems.loc[i,'Picks OUT'] <1):
            dfOrderItems.loc[i,'Picks OUT'] = 0
    #mögliche PAL picks abziehen
        if (dfOrderItems.loc[i,'Picks PAL'] >=1):
            dfOrderItems.loc[i,'Picks OUT'] = (dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks PAL'] * dfOrderItems.loc[i,'PAL'])) * dfOrderItems.loc[i,'OUT']
    #mögliche CS picks abziehen
        if (dfOrderItems.loc[i,'Picks CS'] >=1):
            dfOrderItems.loc[i,'Picks OUT'] = 0#(dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks CS'] * dfOrderItems.loc[i,'CS'])) * dfOrderItems.loc[i,'OUT']
# Picks Gesamt
dfOrderItems['Picks Gesamt'] = dfOrderItems['Picks PAL'] + dfOrderItems['Picks CS'] + dfOrderItems['Picks OUT']
dfOrderItems

,SapOrderNumber,ItemNumber,KVP,Outers,CorrespondingCartons,CorrespondingOuters,CorrespondingPackages,PackagesPerOuter,InternationalArticleNumberForOuter,MaterialNumber,...,InternationalArticleNumberForMastercase,OuterPerMastercase,CS,PAL,OUT,O,Picks PAL,Picks CS,Picks OUT,Picks Gesamt
0,3200817575,000010,27.00,75,3,0,0,10,4031300266243,None,...,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3200817575,000020,27.00,100,4,0,0,10,4031300266250,None,...,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3200817575,000030,99.75,6,1,0,0,5,4031300264843,None,...,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3200817575,000040,99.75,12,2,0,0,5,4031300264850,None,...,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3200817575,000050,33.60,30,1,0,0,6,4031300264812,None,...,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71870,3202235908,000290,90.00,60,3,0,0,6,4031300277652,10158940,...,None,0,6.0,252.0,0.300,18.0,0.0,3.0,0.0,3.0
71871,3202235908,000300,64.00,75,3,0,0,8,4031300267929,10166834,...,None,0,4.8,201.6,0.192,14.4,0.0,3.0,0.0,3.0
71872,3202235908,000310,79.00,25,1,0,0,10,4031300264416,10158853,...,None,0,5.0,210.0,0.200,5.0,0.0,1.0,0.0,1.0
71873,3202235908,000320,76.00,50,2,0,0,10,4031300266427,10158859,...,None,0,5.0,280.0,0.200,10.0,0.0,2.0,0.0,2.0


In [6]:
dfT = pd.merge(dfOrderItems, dfOrder, left_on='SapOrderNumber', right_on='SapOrderNumber', how='left')

dfT


,SapOrderNumber,ItemNumber,KVP,Outers,CorrespondingCartons,CorrespondingOuters,CorrespondingPackages,PackagesPerOuter,InternationalArticleNumberForOuter,MaterialNumber,...,UnloadingListIdentifier,AllSSCCLabelsPrinted,IsSSCCLabelPrintingAllowed,PartnerNo,IDocNumberDESADV,Source,IDocNumberSHPORD,QuantityCheckStatus,QuantityCheckTimestamp,OrderHasConflicts
0,3200817575,000010,27.00,75,3,0,0,10,4031300266243,None,...,20220621_BT ST 9360,False,False,None,None,None,None,0,NaT,False
1,3200817575,000020,27.00,100,4,0,0,10,4031300266250,None,...,20220621_BT ST 9360,False,False,None,None,None,None,0,NaT,False
2,3200817575,000030,99.75,6,1,0,0,5,4031300264843,None,...,20220621_BT ST 9360,False,False,None,None,None,None,0,NaT,False
3,3200817575,000040,99.75,12,2,0,0,5,4031300264850,None,...,20220621_BT ST 9360,False,False,None,None,None,None,0,NaT,False
4,3200817575,000050,33.60,30,1,0,0,6,4031300264812,None,...,20220621_BT ST 9360,False,False,None,None,None,None,0,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71870,3202235908,000290,90.00,60,3,0,0,6,4031300277652,10158940,...,None,False,False,50034577,None,None,0000003645067442,NoDESADVArrived,NaT,False
71871,3202235908,000300,64.00,75,3,0,0,8,4031300267929,10166834,...,None,False,False,50034577,None,None,0000003645067442,NoDESADVArrived,NaT,False
71872,3202235908,000310,79.00,25,1,0,0,10,4031300264416,10158853,...,None,False,False,50034577,None,None,0000003645067442,NoDESADVArrived,NaT,False
71873,3202235908,000320,76.00,50,2,0,0,10,4031300266427,10158859,...,None,False,False,50034577,None,None,0000003645067442,NoDESADVArrived,NaT,False


In [7]:
df

NameError: name 'df' is not defined